<a href="https://colab.research.google.com/github/jeong-wooseok/gnn/blob/main/GNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GNN (Graphic Neural Network) 개요



[참고 : 들어가기 전에 보기  그래프 임배딩 관련 정리](https://medium.com/watcha/%EA%B7%B8%EB%9E%98%ED%94%84-%EC%9E%84%EB%B2%A0%EB%94%A9-%EC%9A%94%EC%95%BD-bc2732048999)<br>
[참고논문 바로가기](https://arxiv.org/ftp/arxiv/papers/1812/1812.08434.pdf) 
<br>


GNN은 이름에서도 알 수 있듯이 그래프에 직접 적용할 수 있는 신경망이다. <br>
GNN은 <b><u> 연결관계와 이웃들의 상태를 이용하여 각 점의 상태를 업데이트(학습)하고 마지막 상태를 통해 예측 업무를 수행</u> </b> 한다. <br>
점 레벨(Node Level), 선 레벨(Edge Level), 그래프 레벨(Graph level)에서 예측 작업에 활용되며, 그 핵심은 점이 이웃과의 연결에 의해 정의된다는 것이다.<br>
<br>
<img src="https://github.com/jeong-wooseok/gnn/blob/main/image/general%20design%20pipeline%20for%20gnn%20model.jpg?raw=true" width="1000" height="500"/>
<br>그래프 타입/스케일을 정하고, Loss Function을 결정하여 인풋하면,<br>
GNN 레이어를 거치며, 노드, 엣지, 그래프 임베딩이 출력된다. 
<br> 설계 시 다음과 같은 단계를 거친다

1. 그래프 구조 확인
* structural scenarios: 분자구조, 물리시스템, 지식그래프 등과 같은 체계화된 그래프 구조
* non-structural scenarios : 그래프로서의 구조화가 잡히지 않은 것은 그래프부터 만들어 줘야 한다. <br>
   (텍스트를 "word"연관관계로 그래프화 등)
   
2. 그래프 타입, 스케일 정의
* 타입 정의 : 방향성 유/무 여부(Directed/Undirected), 점/선의 타입이 동일한지 아닌지 (Homogeneous/Heterogeneous), 계측가능/동적 (Static/Dynamic)
* 규모 정의 : 영역(Criteria)은 디바이스의 연산성능에 따라 변경

3. 손실함수 (Loss Function) 설계
* 노드, 엣지, 그래프 레벨에서의 손실함수 설계
 + 노드레벨 : node classification (노드를 여러 클래스로 분류 진행), node regression (노드의 연산값 예측), node clustering 
 + 엣지레벨 : edge classification (관계 타입 분류), link prediction (두 노드 사이의 관계 예상)
 + 그래프레벨 : e graph classification, graph regression, graph matching
* Supervised / Semi Supervised / Unsupervised Setting 으로 구분해볼 수 있음
 + Supervised : 학습 가능하도록 라벨링 데이터 있음
 + Semi : 라벨링 데이터 소수 보유, 언라벨 노드에 대해 예측값 사용하여 mixed setting
 + Unsupervised : 패턴을 찾음, 노드클러스터링이 대표적
 
4. 컴퓨테이션 모델 설계
* 계산 방식에 따라 propagation modules, sampling modules, pooling modules로 구분.<br>
 + propagation modules : 노드 간 정보를 전이시키며 특성치 및 위상학적 정보를 반영. convolution operator, recurrent operator, skip connection이 있음 <br>
convolution operator, recurrent operator는 이웃으로부터 정보를 집계하는 반면, skip connection은 노드의 전체 상태기록을 활용하여 지나친 획일화 문제를 완화함.
[참고자료](https://ydy8989.github.io/2021-03-03-GAT/)
 + sampling modules : 기존의 방식으로는 메모리 상 한계가 있어서 샘플링을 취하여 적용하는 방식으로 효율성 증대

 


<br><br> 



<img src="https://github.com/jeong-wooseok/gnn/blob/main/image/overview%20of%20computational%20model.jpg?raw=true" width="1000" height="500"/>






## Propagation Module


### Conv. Operator

[관련논문 바로가기](https://arxiv.org/pdf/1901.00596.pdf)<br>

특히 Convolution operator는 GNN에서 가장 흔히 쓰는 방식이다<br>
그래프와 노드의 집합x 가 주어질 때, 각 노드의 은닉표현치(hidden representation)를 그래프 Gconv. 상에 대입하여,<br>
노드간의 관계를 산출한다


<img src="https://github.com/jeong-wooseok/gnn/blob/main/image/c_gnn.jpg?raw=true" width="1000" height="500"/>

ConvGNN : CNN 방식과 비슷하게 그래프를 pooling 하여 Gconv.를 진행하고, 분류화 하는 방식으로 처리할 수도 있다.<br>
결과값은 그래프 표현치를 합친것의 평균값으로 분류화 선택한다.

<img src="https://github.com/jeong-wooseok/gnn/blob/main/image/%ED%99%94%EB%A9%B4%20%EC%BA%A1%EC%B2%98%202022-04-21%20174133.jpg?raw=true" width="1000" height="500"/>

GAE :  Gconv.로 인코딩을 구축하여, 노드 각각의 네트웍임베딩 세트인 Z를 구하고, 이 z행렬 상 서로간의 거리를 구해준다.
 (Z와 전치행렬을 곱한 디코더 층을 통해 노드 네트워크를 산출) 네트워크는 실제 인접 행렬과 재구성된 인접 행렬 간의 불일치를 최소화하여 학습

<img src="https://github.com/jeong-wooseok/gnn/blob/main/image/%ED%99%94%EB%A9%B4%20%EC%BA%A1%EC%B2%98%202022-04-21%20174210.jpg?raw=true" width="1000" height="500"/>

STGNN : 공간/시간 예측.  G-conv층에 1차원의 CNN층이 뒤 따른다. 
G-conv층은 A, X (T)에서 공간적 의존성을 반영하고, 1차원 CNN 계층은 X의 시간축을 통해 슬라이드하여 시간적 종속성 확보. <br>
출력 레이어는 선형 변환하여, 각 노드에 대한 미래의 값등을 예측

<img src="https://github.com/jeong-wooseok/gnn/blob/main/image/%ED%99%94%EB%A9%B4%20%EC%BA%A1%EC%B2%98%202022-04-21%20174245.jpg?raw=true" width="1000" height="500"/>

이전의 GNN 모델들 (~ChebNet)은 graph spectral theory를 바탕으로 graph filtering을 해 왔다는 것을 볼 수 있습니다. <br>
그러나 Kipf & Welling의 GCN논문이 ICLR에 발표되면서, 이후의 모델들은 대부분 spatial domain에서 편안하게 연산을 하기 시작<br>
GCN논문은 spectral domain에서 하던 convolution 연산을 spatial domain에서도 매우 간단하게 할 수 있다!<br>

<img src="https://github.com/jeong-wooseok/gnn/blob/main/image/%ED%99%94%EB%A9%B4%20%EC%BA%A1%EC%B2%98%202022-04-21%20190215.jpg?raw=true" width="1000" height="500"/>




### Recurrent GNN

<br>[관련논문 바로가기](https://arxiv.org/pdf/1901.00596.pdf)<br>

Recurrent Operator는 GNN의 선구적 역할을 진행해왔음. 
상위 수준 노드 표현을 추출하기 위해, 그래프의 노드에 대해 동일한 매개변수 세트를 반복적으로 적용하는 방식으로 산출

입력과 출력이 아래와 같은 함수 $f_w$를 정의하여 점의 상태를 업데이트 <br>
(컨버셔널 모델과는 달리 layer에 동일한 가중치를 적용)<br>
l : layer, 엣지, 특징을 뜻함, x : 노드, 상태를 의미
<br><br>
예를 들어서 <br>
상태(x) : '엄마는 외계인' 이면, 특징(l)은 '밀크베이스' + '초콜릿베이스'+ '초콜릿볼' + ...로 갈라져서 <br>
또 다른 상태(x) : '사랑에 빠진 외계인' 등과 연결됨<br>



$x_n = f_w(l_n, l_{co[n]}, x_{ne[n]}, l_{ne[n]})$

<img src="https://github.com/jeong-wooseok/gnn/blob/main/image/rec_gnn.jpg?raw=true" width="1000" height="500"/>

k번 반복을 통한 업데이트 후 마지막상태($x_n$)와 특징($l_n$)을 사용하여 결과값 $o_n$을 출력한다. <br>
$o_n = g_w(x_n, l_n)$<br><br>

기존 방식은 반복적으로 노드의 은닉상태를 업데이트 하기에 비효율적이며, 각 노드 표현에 중점을 두는 프로젝트이면, 노드의 value가 너무 smoother해졌기에, 타노드와 구분되기에 적정한 정보제공이 어렵다.

- 그래프ESN (GraphESN)(Gallicchio and Micheli, 2010) : 고정된 축약형 인코딩 기능을 사용하고 리드아웃 f만 학습. 다이나믹스저수지(reservoir dynamics)의 축약에 의해 컨버전스. 결과적으로 GNN보다 효율적

- Lagrangian Propagation GNN(LP-GNN)(Tiezzi et al., 2020) : 학습 작업을 Lagrangian 프레임워크의 제약 조건 최적화 문제로 공식화, 고정 소수점에 대한 반복 계산 회피. 

GNN 의 전파 과정(propagation step) 에서의 연산 한계를 감소시키려는 시도는, Gate 메카니즘의 GRU, LSTM 등으로도 발전한다.


<br> recurrent의 gate메카니즘 상 모델 별 오퍼레이터로 적용하는 수식은 다음과 같다.
<br>
![](https://github.com/jeong-wooseok/gnn/blob/main/image/%ED%99%94%EB%A9%B4%20%EC%BA%A1%EC%B2%98%202022-04-21%20143312.jpg?raw=true)
![](https://github.com/jeong-wooseok/gnn/blob/main/image/%ED%99%94%EB%A9%B4%20%EC%BA%A1%EC%B2%98%202022-04-21%20151504.jpg?raw=true)


### Skip connection

더 많은 레이어를 만들 때 더 나은 결과를 얻는 것을 목표로 노드는 k 홉 떨어진 이웃에서 더 많은 정보를 집계. 하지만, 더 깊은 모델은 할 수 없다는 것이 많은 실험에서 관찰
(더 깊은 모델은 더 나쁜 성능을 냄)
- 더 많은 층이 기하급수적으로 증가하는 확장된 이웃 구성원의 잡음 정보 전파
- 오버 스무딩 문제가 발생하기에 모델이 더 깊어질수록 노드간 집계 작업 후에 유사한 표현을 갖는 경향

"Skip connection”기법으로 깊이 있는 GNN 모델에 적용

## 기타 모듈

### Sampling Module

### Pooling module

# GNN 접근 방법

## 그래프 타입/규모 별 변수

## 학습 셋팅 별 변수

## GNN 설계 절차 

1. Find graph structure
2. Specify graph type and scale.
3. Design loss function.
4. Build model using computational modules.


## GNN 분석방법 
1. 이론적 측면
 * Graph signal processing
 * Generalization
 * Expressivity
 *  

2. 

[기타 : GNN 관련 분야 별 논문 정리](https://github.com/thunlp/GNNPapers#analysis)

[참고 : GNN 관련 쉽게 이해할 수 있는 글](https://medium.com/watcha/gnn-%EC%86%8C%EA%B0%9C-%EA%B8%B0%EC%B4%88%EB%B6%80%ED%84%B0-%EB%85%BC%EB%AC%B8%EA%B9%8C%EC%A7%80-96567b783479) <br>

[참고 : GraphCNN 이해하기 "1편](https://ahjeong.tistory.com/14)
["2편](https://ahjeong.tistory.com/15)
